In [361]:
import pandas as pd
import pathlib
import numpy as np

In [362]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [363]:
df = pd.read_excel(file, skiprows=1, header=None)
qgis_table = pd.read_excel(file, sheet_name=1, header=0, usecols=["AssANode","AssBNode", "ID"])

In [364]:
df=df[df.iloc[:,0] == "route"]

In [365]:
lst = df.to_string(header=False, index=False).split()

In [366]:
lst

['route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74301.00',
 '74095.00',
 '746.90',
 'NaN',
 'NaN',
 'NaN',
 'route',
 '74491.0',
 '74490.0',
 '74076.00',
 '746.40',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'route',
 '74491.0',
 '74490.0',
 '74076.00',
 '741.40',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74638.00',
 '74094.00',
 '74365.00',
 '748.03',
 'NaN',
 'NaN',
 'route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74301.00',
 '74095.00',
 '74336.00',
 '742.54',
 'NaN',
 'NaN',
 'route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74301.00',
 '74095.00',
 '743.18',
 'NaN',
 'NaN',
 'NaN',
 'route',
 '74690.0',
 '74095.0',
 '74301.00',
 '74320.00',
 '74638.00',
 '74094.00',
 '74

In [367]:
lst=list(filter(lambda x: "NaN" not in x, lst))

In [368]:
lst

['route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74301.00',
 '74095.00',
 '746.90',
 'route',
 '74491.0',
 '74490.0',
 '74076.00',
 '746.40',
 'route',
 '74491.0',
 '74490.0',
 '74076.00',
 '741.40',
 'route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74638.00',
 '74094.00',
 '74365.00',
 '748.03',
 'route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74301.00',
 '74095.00',
 '74336.00',
 '742.54',
 'route',
 '74491.0',
 '74490.0',
 '74077.00',
 '74801.00',
 '74637.00',
 '74093.00',
 '74320.00',
 '74301.00',
 '74095.00',
 '743.18',
 'route',
 '74690.0',
 '74095.0',
 '74301.00',
 '74320.00',
 '74638.00',
 '74094.00',
 '74022.00',
 '74804.00',
 '74800.00',
 '74077.00',
 '74490.00',
 '744.91',
 'route',
 '74690.0',
 '74095.0',
 '74301.00',
 '74320.00',
 '74638.00',
 '74094.00',
 '74022.00',
 '74804.00',
 '74800.00',
 '74077.00',
 '74801.00',
 

In [370]:
matrix_reloaded = []
count=-1
for i in range(len(lst)):
    if lst[i] == "route":
        matrix_reloaded.append([])
        count+=1
        continue
    print(lst[i])
    matrix_reloaded[count].append(int(float(lst[i])))

74491.0
74490.0
74077.00
74801.00
74637.00
74093.00
74320.00
74301.00
74095.00
746.90
74491.0
74490.0
74076.00
746.40
74491.0
74490.0
74076.00
741.40
74491.0
74490.0
74077.00
74801.00
74637.00
74093.00
74320.00
74638.00
74094.00
74365.00
748.03
74491.0
74490.0
74077.00
74801.00
74637.00
74093.00
74320.00
74301.00
74095.00
74336.00
742.54
74491.0
74490.0
74077.00
74801.00
74637.00
74093.00
74320.00
74301.00
74095.00
743.18
74690.0
74095.0
74301.00
74320.00
74638.00
74094.00
74022.00
74804.00
74800.00
74077.00
74490.00
744.91
74690.0
74095.0
74301.00
74320.00
74638.00
74094.00
74022.00
74804.00
74800.00
74077.00
74801.00
74810.00
748.11
74690.0
74095.0
74301.00
74320.00
74638.00
74094.00
74022.00
74807.00
748.08
74690.0
74095.0
74301.00
74320.00
74638.00
74094.00
74022.00
74804.00
74800.00
74077.00
74490.00
74076.00
746.4
74690.0
74095.0
74301.00
74320.00
74638.00
74094.00
74022.00
74804.00
74800.00
74077.00
74490.00
74076.00
741.4
74690.0
74095.0
74301.00
74320.00
74638.00
74094.00
7436

ValueError: could not convert string to float: '74810+'

In [ ]:
Trying to convert strings of decimal value to integers so they can be matched in the gis df

In [ ]:
res=[]
for i in range(len(matrix_reloaded)):
    res.append([])
    for j in range(len(matrix_reloaded[i])-1):
        res[i].append(matrix_reloaded[i][j] +">" + matrix_reloaded[i][j+1])

In [ ]:
qgis_table

In [ ]:
qgis_table[qgis_table["AssBNode"].str.contains('74491>74490')]